In [2]:
# https://www.kaggle.com/datasets/dhruvildave/en-fr-translation-dataset

from typing import NamedTuple, Generator
import pandas as pd
import torch
import torch.nn.functional as F
from torch import Tensor, optim, nn
from tqdm import tqdm
from transformers import AutoTokenizer
from src.transformer import Transformer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class PairedSentences(NamedTuple):
    fr: str
    en: str

class ListPairedSentences(NamedTuple):
    fr: list[str]
    en: list[str]

    def __getitem__(self, index: int) -> PairedSentences:
        return PairedSentences(self.fr[index], self.en[index])

class TrainingBatch(NamedTuple):
    input_ids: Tensor
    """In our case french"""
    
    encoder_mask: Tensor
    """basically padding tokens"""
    
    output_ids: Tensor
    """In our case english"""
    
    decoder_mask: Tensor
    """Padding tokens, don't forget to add causal mask during training"""

    def __repr__(self):
        return f"TrainingBatch(x.shape={self.input_ids.shape}, y.shape={self.output_ids.shape})"

In [14]:
class Processor:
    def __init__(self, sequence_length: int, tokenizer_name: str) -> None:
        self._tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        self._seq_length = sequence_length
        
    @property
    def tokenizer(self) -> AutoTokenizer:
        return self._tokenizer

    @property
    def sequence_length(self) -> int:
        return self._seq_length

    @property
    def vocab_size(self) -> int:
        return self._tokenizer.vocab_size

    def tokenize(self, text: str, padding: str="max_length", truncation:bool=True, extra: int = 0):
        return self._tokenizer(
            text,
            return_tensors="pt",
            max_length=self._seq_length + extra,
            padding=padding,
            truncation=truncation)

    def decode(self, token_ids: Tensor, **kwargs) -> str:
        return self._tokenizer.decode(token_ids, **kwargs)

    def make_batch(self, paired_sentences: ListPairedSentences, dtype=torch.float32) -> TrainingBatch:
        # Tokenize each sentence in the 'fr' and 'en' lists
        fr_sentences = [self.tokenize(sentence) for sentence in paired_sentences.fr]
        en_sentences = [self.tokenize(sentence, extra=1) for sentence in paired_sentences.en]

        # Stack tokenized tensors for batching
        X_batch = torch.stack([x['input_ids'].squeeze(0) for x in fr_sentences])
        Y_batch = torch.stack([y['input_ids'].squeeze(0) for y in en_sentences])

        # Create encoder and decoder padding mask: 1 for real tokens, 0 for padding
        encoder_mask = torch.stack([x['attention_mask'].squeeze(0) for x in fr_sentences]) \
            .unsqueeze(1).unsqueeze(2)
        decoder_mask = torch.stack([y['attention_mask'].squeeze(0) for y in en_sentences]) \
            .unsqueeze(1).unsqueeze(2)

        return TrainingBatch(
            input_ids=X_batch,
            output_ids=Y_batch,
            encoder_mask=encoder_mask.to(dtype),
            decoder_mask=decoder_mask.to(dtype))

def get_first_masked_token(mask: torch.Tensor) -> Tensor:
    squeezed_mask = mask.squeeze(1).squeeze(1) # mask is shaped (bs, 1, 1, sequence_length)
    first_masked_indices = (squeezed_mask == 0).int().argmax(dim=1)
    first_masked_indices[squeezed_mask.sum(dim=1) == squeezed_mask.size(1)] = squeezed_mask.size(1)
    return first_masked_indices.to(dtype=torch.int32)

def mask_last_token(current_mask: torch.Tensor) -> torch.Tensor:
    first_masked_indices = get_first_masked_token(current_mask) # get the index of first masked token
    last_token_indices = torch.clamp(first_masked_indices - 1, min=0) # to avoid negative indices
    current_mask[torch.arange(current_mask.size(0)), 0, 0, last_token_indices] = 0 # set the last 1 token to 0
    return current_mask

In [15]:
def get_page(csv_path: str, page: int, rows_per_page: int):
    return pd.read_csv(csv_path, skiprows = 1 + page * rows_per_page, nrows=rows_per_page, header=None, names=["en", "fr"])

def make_generator(csv_path: str, rows_per_page: int) -> Generator[ListPairedSentences, None, None]:
    i = 0
    while True:
        page = get_page(csv_path, i, rows_per_page)
        fr_sentences = page["fr"].to_list()
        en_sentences = page["en"].to_list()
        yield ListPairedSentences(fr_sentences, en_sentences)
        i += 1

In [25]:
dtype=torch.bfloat16
csv_path = "archive/en-fr.csv"
processor = Processor(200, "bert-base-uncased")
model = Transformer(vocab_size=processor.vocab_size, max_sequence_len=processor.sequence_length, d_model=256).to(DEVICE).to(dtype)

# opt and loss
learning_rate = 1e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss(ignore_index=processor.tokenizer.pad_token_id).to(DEVICE)

# training loop qty
num_epochs = 500
batch_size = 32
break_at = 10
# get_num_steps(csv_path, batch_size) running this function is too long ... just use the cached value
num_steps = 22520376 // batch_size



In [26]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    model.to(DEVICE)
    epoch_loss = 0.0
    loop_error = 0.0

    # Reset generator at the start of each epoch
    progress_bar = tqdm(make_generator(csv_path, batch_size), total=num_steps)
    progress_bar.set_description(f"Epoch {epoch + 1}")

    for step, raw_batch in enumerate(progress_bar, start=1):
        try:
            training_batch = processor.make_batch(raw_batch, dtype)        # Converts batch to `TrainingBatch` format
            input_ids = training_batch.input_ids.to(DEVICE)            # Encoder input (source sentence)
            target_ids = training_batch.output_ids.to(DEVICE)          # Decoder target sequence
            encoder_mask = training_batch.encoder_mask.to(DEVICE)   # Mask for encoder

            # adjust decoder input
            decoder_input_ids = target_ids[:, :-1]
            target_ids_flat = target_ids[:, 1:].contiguous().view(-1)

            # adjust decoder mask too
            decoder_mask = training_batch.decoder_mask[:, :, :, :-1].to(DEVICE)
            
            # We need to create a causal mask too
            seq_len = decoder_input_ids.shape[1]
            causal_mask = torch.tril(torch.ones((seq_len, seq_len))).to(DEVICE).to(dtype)
            
            # final decoder mask as prod of padding * causal
            final_decoder_mask = decoder_mask * causal_mask.unsqueeze(0)

        except Exception as e:
            print(f"Error in batch {step}: {e}")
            loop_error += 1
            continue
        
        # change learning rate
        # new_lr = 512 ** (-1.5) * (epoch * 1000 + step + 1) ** (-0.5)
        # for param_group in optimizer.param_groups:
        #     param_group['lr'] = new_lr

        # Forward pass
        optimizer.zero_grad()
        output_probs = model(
            input_ids,
            decoder_input_ids,
            encoder_mask=encoder_mask,
            decoder_mask=final_decoder_mask)

        # flatten target and outputprobs to compute cce loss
        output_probs_flat = output_probs.view(-1, output_probs.size(-1))

        # Calculate the loss
        loss = loss_fn(output_probs_flat, target_ids_flat)
        loss.backward()
        optimizer.step()

        # Track loss
        epoch_loss += loss.item()

        # Optionally, print progress
        progress_bar.set_postfix_str(f"lr : {learning_rate:.4f} ; "
                                     f"epoch loss : {epoch_loss / step:.4f} ; "
                                     f"loop error : {loop_error}")
        
        
        if step == break_at: # train only on a subset for now
            break

    # with torch.no_grad():
    #     sample_output = output_probs.argmax(dim=-1)
    #     print("Predicted tokens:", sample_output[0, :10].squeeze())  
    #     print("Target tokens:   ", target_ids[0, 1:11].squeeze())  

    # Print average loss per epoch
    print(f"Epoch [{epoch+1}/{num_epochs}] completed, Average Loss: {epoch_loss / num_steps:.4f}")

Epoch 1:   0%|          | 9/703761 [00:00<20:24:10,  9.58it/s, lr : 0.0001 ; epoch loss : 9.6438 ; loop error : 0.0] 


Epoch [1/500] completed, Average Loss: 0.0001


Epoch 2:   0%|          | 9/703761 [00:00<19:55:57,  9.81it/s, lr : 0.0001 ; epoch loss : 9.1250 ; loop error : 0.0]


Epoch [2/500] completed, Average Loss: 0.0001


Epoch 3:   0%|          | 9/703761 [00:00<20:11:07,  9.68it/s, lr : 0.0001 ; epoch loss : 8.8250 ; loop error : 0.0]


...

Epoch [293/500] completed, Average Loss: 0.0000


Epoch 294:   0%|          | 9/703761 [00:00<19:09:21, 10.21it/s, lr : 0.0001 ; epoch loss : 1.5625 ; loop error : 0.0]


Epoch [294/500] completed, Average Loss: 0.0000


Epoch 295:   0%|          | 9/703761 [00:00<16:46:07, 11.66it/s, lr : 0.0001 ; epoch loss : 1.5052 ; loop error : 0.0]


KeyboardInterrupt: 

In [27]:
@torch.no_grad()
def infer(model: nn.Module, 
          tokenerizer: Processor, 
          french_sentence: str, 
          max_length: int | None = None, 
          skip_special_tokens: bool = True) -> str:
    # first model as eval (we don't train here)
    model.eval()
    sequence_length = max_length if max_length else tokenerizer.sequence_length

    # Tokenize the input sequence in french
    tokens = tokenerizer.tokenize(french_sentence)

    # create the encoder input and mask
    encoder_ids = tokens["input_ids"].int().to(DEVICE)
    encoder_mask = tokens["attention_mask"].unsqueeze(0).unsqueeze(0).to(DEVICE)

    # initialize decoder mask and input
    decoder_mask = torch.zeros((1, 1, 1, tokenerizer.sequence_length)).to(DEVICE)
    decoder_mask[0, 0, 0, 0] = 1 # unmask the start of sequence token
    target_ids = torch.zeros((1, tokenerizer.sequence_length)).int().to(DEVICE)
    target_ids[0, 0] = tokenerizer.tokenizer.cls_token_id # start of sequence

    # loop to generate output ids
    for idx in range(1, sequence_length):
        output_probs: Tensor = model(
            encoder_ids,
            target_ids,
            encoder_mask=encoder_mask,
            decoder_mask=None)

        # select next token ID with the highest probability
        next_token_id = output_probs.argmax(dim=-1)[0, idx]
        target_ids[0, idx] = next_token_id
        decoder_mask[0, 0, 0, idx] = 1 # unmask the generated token

        # early stop when encounter sep_token_id
        if next_token_id.item() == tokenerizer.tokenizer.sep_token_id:
            break

    return tokenerizer.decode(target_ids[0], skip_special_tokens=skip_special_tokens)

In [37]:
sentence = make_generator(csv_path, 1)
print(next(sentence))
infer(model, processor, "Wow what the actual fuck ?")

ListPairedSentences(fr=['Il a transformé notre vie | Il a transformé la société | Son fonctionnement | La technologie, moteur du changement Accueil | Concepts | Enseignants | Recherche | Aperçu | Collaborateurs | Web HHCC | Ressources | Commentaires Musée virtuel du Canada'], en=['Changing Lives | Changing Society | How It Works | Technology Drives Change Home | Concepts | Teachers | Search | Overview | Credits | HHCC Web | Reference | Feedback Virtual Museum of Canada Home Page'])


'.......................................................................................................................................................................................................'